In [1]:
import pandas as pd
import seaborn as sns
from datetime import date

In [2]:
en_to_vn = {
    "WHOLE INDUSTRY": "Toàn ngành công nghiệp",
    "Mining and quarying": "Khai khoáng",
    "Mining of coal and lignite": "Khai thác than cứng và than non",
    "Extraction of crude petroleum and nutural gas": "Khai thác dầu thô và khí đốt tự nhiên",
    "Mining of metal ores": "Khai thác quặng kim loại",
    "Other mining and quarrying (stone, sand and clay)": "Khai khoáng khác",
    "Mining support service activities": "Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng",
    "Manufacturing": "Công nghiệp chế biến, chế tạo",
    "Manufacture of food products": "Sản xuất, chế biến thực phẩm",
    "Manufacture of beverages": "Sản xuất đồ uống",
    "Manufacture of tobacco products": "Sản xuất sản phẩm thuốc lá",
    "Manufacture of textiles": "Dệt",
    "Manufacture of wearing apparel": "Sản xuất trang phục",
    "Manufacture of leather and related products": "Sản xuất da và các sản phẩm có liên quan",
    "Manufacture of wood and of products of wood and cork": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "Manufacture of paper and paper products": "Sản xuất giấy và sản phẩm từ giấy",
    "Printing and reproduction of recorded media": "In, sao chép bản ghi các loại",
    "Manufacture of coke and refined petroleum products": "Sản xuất than cốc, sản phẩm dầu mỏ tinh chế",
    "Manufacture of chemicals and chemical products": "Sản xuất hoá chất và sản phẩm hoá chất",
    "Manufacture of pharmaceuticals, medicinal chemical and botanical products": "Sản xuất thuốc, hoá dược và dược liệu",
    "Manufacture of rubber and plastics products": "Sản xuất sản phẩm từ cao su và plastic",
    "Manufacture of other non-metallic mineral products": "Sản xuất sản phẩm từ khoáng phi kim loại khác",
    "Manufacture of basic metals": "Sản xuất kim loại",
    "Manufacture of fabricated metal products, except machinery and equipment": "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)",
    "Manufacture of computer, electronic and optical products": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "Manufacture of electrical equipment": "Sản xuất thiết bị điện",
    "Manufacture of machinery and equipment n.e.c": "Sản xuất máy móc, thiết bị chưa được phân vào đâu",
    "Manufacture of motor vehicles; trailers and semi-trailers": "Sản xuất xe có động cơ",
    "Manufacture of other transport equipment": "Sản xuất phương tiện vận tải khác",
    "Manufacture of furniture": "Sản xuất giường, tủ, bàn, ghế",
    "Other manufacturing": "Công nghiệp chế biến, chế tạo khác",
    "Repair and installation of machinery and equipment": "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị",
    "Electricity, gas, steam and air conditioning supply": "Sản xuất và phân phối điện",
    "Water supply; sewerage, waste management and remediation activities": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Water collection, treatment and supply": "Khai thác, xử lý và cung cấp nước",
    "Sewerage and sewer treatment activities": "Thoát nước và xử lý nước thải",
    "Waste collection, treatment and disposal activities; materials recovery": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
}


def find_sheet_name(sheet_names):
    iip_sheet_names = list(filter(lambda x: "iip" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(filter(lambda x: "thang" in x.lower(), iip_sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    if len(sheet_names) == 43:
        return "7"

    iip_sheet_names = list(filter(lambda x: "gtcn" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    raise RuntimeError


def get_ipp_2023(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all")

    if len(df.columns) == 5:
        df = df.drop(columns=[df.columns[1], df.columns[2]])
    elif len(df.columns) == 6:
        df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[5]])
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1], df.columns[2]])
        df["A"] = df[df.columns[1]]
    elif len(df.columns) == 4:
        df = df.drop(columns=[df.columns[1]])
    elif len(df.columns) == 3 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1]])
        df["A"] = df[df.columns[1]]
    elif len(df.columns) == 3:
        df = df.drop(columns=[df.columns[1]])
        df["A"] = df[df.columns[1]]
        raise RuntimeError(date)

    df[df.columns[0]] = (
        df[df.columns[0]].replace(r"\s+", " ", regex=True).replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df_iip_per_month = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "Toàn ngành công nghiệp")
                    | (df[df.columns[0]] == "WHOLE INDUSTRY")
                    | (df[df.columns[0]] == "TOÀN NGÀNH CÔNG NGHIỆP")
                    | (df[df.columns[0]] == "TOÀN QUỐC")
                ].index[0]
            )
        )
    )

    df_iip_per_month.columns = [
        "Ngành",
        "Tháng này so với cùng kỳ năm trước",
        "Từ đầu năm đến nay so với cùng kỳ năm trước",
    ]

    if len(df_iip_per_month["Ngành"][df_iip_per_month["Ngành"] == "WHOLE INDUSTRY"].index) > 0:
        df_iip_per_month["Ngành"] = df_iip_per_month["Ngành"].map(en_to_vn)

    df_iip_per_month = df_iip_per_month[
        pd.to_numeric(
            df_iip_per_month["Tháng này so với cùng kỳ năm trước"], errors="coerce"
        ).notnull()
    ]
    df_iip_per_month = df_iip_per_month[
        pd.to_numeric(
            df_iip_per_month["Từ đầu năm đến nay so với cùng kỳ năm trước"], errors="coerce"
        ).notnull()
    ]

    df_iip_per_month["Tháng này so với cùng kỳ năm trước"] = (
        df_iip_per_month["Tháng này so với cùng kỳ năm trước"] - 100
    ).astype(float)

    df_iip_per_month["Từ đầu năm đến nay so với cùng kỳ năm trước"] = (
        df_iip_per_month["Từ đầu năm đến nay so với cùng kỳ năm trước"] - 100
    ).astype(float)

    df_iip_per_month["Date"] = date
    df_iip_per_month["Date"] = pd.to_datetime(df_iip_per_month["Date"], dayfirst=False)

    df_iip_per_month = df_iip_per_month[
        df_iip_per_month["Từ đầu năm đến nay so với cùng kỳ năm trước"].notna()
    ]

    df_iip_per_month = df_iip_per_month[
        df_iip_per_month["Tháng này so với cùng kỳ năm trước"].notna()
    ]

    df_iip_per_month = df_iip_per_month.replace(
        {"rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải"}
    )

    return df_iip_per_month


xl = pd.ExcelFile("datas/2023-01-29-02-So-lieu-01.2023-1.xlsx")
get_ipp_2023(xl, find_sheet_name(xl.sheet_names), "2023-01-29").head(n=15)

,Ngành,Tháng này so với cùng kỳ năm trước,Từ đầu năm đến nay so với cùng kỳ năm trước,Date
5,Toàn ngành công nghiệp,-8.04,-8.04,2023-01-29
6,Khai khoáng,-4.86,-4.86,2023-01-29
7,Khai thác than cứng và than non,-8.82,-8.82,2023-01-29
8,Khai thác dầu thô và khí đốt tự nhiên,-2.53,-2.53,2023-01-29
9,Khai thác quặng kim loại,2.81,2.81,2023-01-29
10,Khai khoáng khác,-6.73,-6.73,2023-01-29
11,Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng,-10.70,-10.70,2023-01-29
12,"Công nghiệp chế biến, chế tạo",-9.10,-9.10,2023-01-29
13,"Sản xuất, chế biến thực phẩm",-1.99,-1.99,2023-01-29
14,Sản xuất đồ uống,17.46,17.46,2023-01-29


In [3]:
import os
import contextlib

dfs_iip_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    with contextlib.suppress(Exception):
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_iip = get_ipp_2023(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_iip_origin.append(df_iip)

In [4]:
dfs_iip = pd.concat(dfs_iip_origin, ignore_index=True, axis=0)
dfs_iip["Date"] = pd.to_datetime(dfs_iip["Date"], dayfirst=False)
# dfs_iip[dfs_iip["Date"] > "2014-01-01"]["Ngành"].unique()

In [5]:
convert_nganh = {
    "Toàn ngành công nghiệp": "Toàn ngành công nghiệp",
    "Khai khoáng": "Khai khoáng",
    "Khai thác than cứng và than non": "Khai thác than cứng và than non",
    "Khai thác dầu thô và khí đốt tự nhiên": "Khai thác dầu thô và khí đốt tự nhiên",
    "Khai thác quặng kim loại": "Khai thác quặng kim loại",
    "Khai khoáng khác": "Khai khoáng khác",
    "Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng": "Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng",
    "Công nghiệp chế biến, chế tạo": "Công nghiệp chế biến, chế tạo",
    "Sản xuất, chế biến thực phẩm": "Sản xuất, chế biến thực phẩm",
    "Sản xuất đồ uống": "Sản xuất đồ uống",
    "Sản xuất sản phẩm thuốc lá": "Sản xuất sản phẩm thuốc lá",
    "Dệt": "Dệt",
    "Sản xuất trang phục": "Sản xuất trang phục",
    "Sản xuất da và các sản phẩm có liên quan": "Sản xuất da và các sản phẩm có liên quan",
    "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "Sản xuất giấy và sản phẩm từ giấy": "Sản xuất giấy và sản phẩm từ giấy",
    "In, sao chép bản ghi các loại": "In, sao chép bản ghi các loại",
    "Sản xuất than cốc, sản phẩm dầu mỏ tinh chế": "Sản xuất than cốc, sản phẩm dầu mỏ tinh chế",
    "Sản xuất hoá chất và sản phẩm hoá chất": "Sản xuất hoá chất và sản phẩm hoá chất",
    "Sản xuất thuốc, hoá dược và dược liệu": "Sản xuất thuốc, hoá dược và dược liệu",
    "Sản xuất sản phẩm từ cao su và plastic": "Sản xuất sản phẩm từ cao su và plastic",
    "Sản xuất sản phẩm từ khoáng phi kim loại khác": "Sản xuất sản phẩm từ khoáng phi kim loại khác",
    "Sản xuất kim loại": "Sản xuất kim loại",
    "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)": "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)",
    "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "Sản xuất thiết bị điện": "Sản xuất thiết bị điện",
    "Sản xuất máy móc, thiết bị chưa được phân vào đâu": "Sản xuất máy móc, thiết bị chưa được phân vào đâu",
    "Sản xuất xe có động cơ": "Sản xuất xe có động cơ",
    "Sản xuất phương tiện vận tải khác": "Sản xuất phương tiện vận tải khác",
    "Sản xuất giường, tủ, bàn, ghế": "Sản xuất giường, tủ, bàn, ghế",
    "Công nghiệp chế biến, chế tạo khác": "Công nghiệp chế biến, chế tạo khác",
    "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị": "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị",
    "Sản xuất và phân phối điện": "Sản xuất và phân phối điện",
    "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Khai thác, xử lý và cung cấp nước": "Khai thác, xử lý và cung cấp nước",
    "Thoát nước và xử lý nước thải": "Thoát nước và xử lý nước thải",
    "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn, ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "Sửa chữa, bảo dưỡng và lắp đặt máy móc và thiết bị": "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị",
    "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí": "Sản xuất và phân phối điện",
    "rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "(trừ máy móc, thiết bị)": "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)",
    "và sản phẩm quang học": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "Sản xuất chế biến thực phẩm": "Sản xuất, chế biến thực phẩm",
    "Sản xuất thuốc lá": "Sản xuất sản phẩm thuốc lá",
    "Hoạt độg thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "TOÀN NGÀNH CÔNG NGHIỆP": "Toàn ngành công nghiệp",
    "và xử lý rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "HĐ thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "Sản xuất sản phẩm điện tử, máy vi tưnh và sản phẩm quang học": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "Công nghiệp chế biến , chế tạo": "Công nghiệp chế biến, chế tạo",
    "TOÀN QUỐC": "Toàn ngành công nghiệp",
}

dfs_iip["Ngành"] = dfs_iip["Ngành"].map(convert_nganh)
dfs_iip

,Ngành,Tháng này so với cùng kỳ năm trước,Từ đầu năm đến nay so với cùng kỳ năm trước,Date
0,Toàn ngành công nghiệp,5.76,1.46,2023-12-29
1,Khai khoáng,-12.81,-3.90,2023-12-29
2,Khai thác than cứng và than non,-11.01,-1.61,2023-12-29
3,Khai thác dầu thô và khí đốt tự nhiên,-18.97,-5.69,2023-12-29
4,Khai thác quặng kim loại,10.00,9.51,2023-12-29
...,...,...,...,...
3954,"Sản xuất giường, tủ, bàn, ghế",6.94,6.94,2014-01-29
3955,Sản xuất và phân phối điện,4.40,4.40,2014-01-29
3956,Cung cấp nước; hoạt động quản lý và xử lý rác ...,8.99,8.99,2014-01-29
3957,"Khai thác, xử lý và cung cấp nước",9.72,9.72,2014-01-29


In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_iip[
    (
        (dfs_iip["Ngành"] == "Toàn ngành công nghiệp")
        | (dfs_iip["Ngành"] == "Khai khoáng")
        | (dfs_iip["Ngành"] == "Công nghiệp chế biến, chế tạo")
        | (dfs_iip["Ngành"] == "Sản xuất và phân phối điện")
    )
    & (dfs_iip["Date"] > "2015-01-01")
]

fig = px.area(df_show, x="Date", y="Tháng này so với cùng kỳ năm trước", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_iip[
    (
        (dfs_iip["Ngành"] == "Sản xuất kim loại")
        | (
            dfs_iip["Ngành"]
            == "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)"
        )
        | (dfs_iip["Ngành"] == "Dệt")
        # | (dfs_iip["Ngành"] == "Sản xuất trang phục")
        | (
            dfs_iip["Ngành"]
            == "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học"
        )
        | (dfs_iip["Ngành"] == "Sản xuất thiết bị điện")
        # | (dfs_iip["Ngành"] == "Khai thác dầu thô và khí đốt tự nhiên")
        | (dfs_iip["Ngành"] == "Sản xuất sản phẩm từ cao su và plastic")
        | (dfs_iip["Ngành"] == "Sản xuất hoá chất và sản phẩm hoá chất")
    )
    & (dfs_iip["Date"] > "2022-01-01")
]

fig = px.area(df_show, x="Date", y="Tháng này so với cùng kỳ năm trước", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

